In [2]:
import keras, os, pickle, re, sklearn, string, tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.layers import Embedding
from keras.optimizers import Adadelta, adam
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils.vis_utils  import plot_model
from sklearn import preprocessing
from keras import optimizers

print('Keras version: \t\t%s' % keras.__version__)
print('Scikit version: \t%s' % sklearn.__version__)
print('TensorFlow version: \t%s' % tensorflow.__version__)


Using TensorFlow backend.


Keras version: 		2.0.9
Scikit version: 	0.19.1
TensorFlow version: 	1.4.1


In [3]:
import nltk
from nltk.corpus import stopwords
from tika import parser
from sklearn.metrics import classification_report



In [3]:
def LoadData_nontech():
    label_nontech=[]
    i=0
    for foldername in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level3/nontech"):
        for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level3/nontech/"+foldername):
    
            try:
                print (i, foldername, file)
                parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level3/nontech/"+foldername+"/"+file)

                resume_contents=clean_doc(parsedPDF['content'])
            #Data = resume_contents.encode('utf-8')    
                label_nontech.append((resume_contents,foldername))
            except UnicodeEncodeError:
                print ('Unicode error:', file)
            i=i+1
    #print (label_resume)
    return(label_nontech)
        

In [4]:
#Preprocessing
def clean_doc(doc):
    """
    Cleaning a document by several methods:
        - Lowercase
        - Removing whitespaces
        - Removing stopwords
        - Removing punctuations
        
    """
    stop_words = set(stopwords.words('english'))
    
    # Lowercase
    doc = doc.lower()
   
    tokens = doc.translate({ord(c):"" for c in "\u200b\uf020\u2028\xa0\uf0e0\uf095\uf041\uf0e1\uf0b7\xad"})
    tokens = tokens.translate({ord(c):" " for c in "[):,·](;•●■♦▪"})
    tokens = tokens.translate({ord(c):"f" for c in "�"})
    
    # Removing multiple whitespaces
    tokens = re.sub(r"\?", " \? ", tokens)
    
    # Split in tokens
    tokens = tokens.split()
    
    # Remove Stopwords
    tokens = [w for w in tokens if not w in stop_words]
    # Remove punctuation
    tokens = [w.translate(str.maketrans('', '', string.punctuation)) for w in tokens]
    
    return ' '.join(tokens)

In [5]:
label_nontech=LoadData_nontech()

0 marketing Bhaskar_Singh_Kolkata_15.00_yrs.doc
1 marketing Samuel_Raja_Velapula_Bengaluru___Bangalore_5.01_yrs.docx
2 marketing Pranshu_Gupta_Delhi_10.00_yrs.doc
3 marketing Rachita_Vig_Pune_0.00_yrs.docx
4 marketing Mrinal_Tiwari_Bhubaneshwar_1.00_yrs.pdf
5 marketing Brijesh_Pradhan_Chennai_13.00_yrs.doc
6 marketing Parvez_Ali_Hyderabad___Secunderabad_1.10_yrs.pdf
7 marketing Ravi_Anant_Sharma_Bengaluru___Bangalore_4.09_yrs.pdf
8 marketing shabaiz_khan_Bengaluru___Bangalore_4.08_yrs.doc
9 marketing Bal_Mukund_Thakur_Delhi_9.00_yrs.doc
10 marketing Sujith N V.doc
11 marketing Chandan_Das_Hyderabad___Secunderabad_10.01_yrs.doc
12 marketing Johny__Mumbai_10.00_yrs.docx
13 marketing Chitrakala_Natarajan_Chennai_7.06_yrs.docx
14 marketing Pritam_Barik_Gurgaon_3.06_yrs.pdf
15 marketing Shaleen__Bengaluru___Bangalore_10.01_yrs.docx
16 marketing shashi_kant_upadhyay_Chennai_3.01_yrs.docx
17 marketing pavan_praneeth_nallamilli_Bengaluru___Bangalore_2.04_yrs.docx
18 marketing Irfan_Shaik_Hyder

173 hr RitaMathews[0_7].pdf
174 hr TeoPaul[4_5].docx
175 hr SimonDavisS[8_0].doc
176 hr ShelinKGeorge[6_0].pdf
177 hr GeorginaKurian[11_0].doc
178 hr MaheshMenon[15_0].doc
179 hr SureshKumarM[7_0].docx
180 hr SusindharanGB[10_0].docx
181 hr Sudhaya[11_0].pdf
182 hr NayanaDevadas[3_9].pdf
183 hr PARUCHURIRAMU[6_0].docx
184 hr SharonAnbudaiyan[1_1].doc
185 hr ShijuBhaskaran[14_0].doc
186 hr SFelixJayabalan[11_0].doc
187 hr PratapChandranK[9_0].doc
188 hr Neenu_Saseendran_Cochin___Kochi___Ernakulam_2.02_yrs.docx
189 hr PavithraB[5_5].pdf
190 hr MeenuSunil[1_1].docx
191 hr PragilP[3_0].docx
192 hr Jijokuruvilla[7_0].pdf
193 hr PradeepK[4_0].docx
194 hr KabilanSP[7_0].doc
195 hr SajeeshPV[6_0].doc
196 hr JohnsonS[17_0].docx
197 hr SwarnalathaV[4_0].docx
198 hr Nisha_Shankar.pdf
199 hr PrasadDS[13_0].docx
200 hr SrinivasKuchimanchi[15_0].docx
201 hr Swapna[10_0].doc
202 hr JayachandranR[19_0].docx
203 hr SHYAMKRISHNANS[3_0].docx
204 hr MANOJP[12_0].pdf
205 hr JERRYJPAUL[4_5].pdf
206 hr Neera

420 seo Ranjeet_kumar_Noida_3.07_yrs.doc
421 seo Jayshree_Sasane_upadhyay_Pune_7.00_yrs.doc
422 seo vishnu_kumar_Delhi_2.07_yrs.docx
423 seo Divya[2_0].doc
424 seo BhuvaneswariA[1_1].docx
425 seo LakshmiRDevi[5_3].pdf
426 seo SSasikiran[2_6].doc
427 seo RajashriGHegde[1_10].doc
428 seo AbhishekC[2_3].doc
429 seo PShriVanadhi[1_0].doc
430 seo Pawan_Gupta_Gurgaon_3.11_yrs.doc
431 seo ashok__Faridabad_1.10_yrs.docx
432 seo Anil_Pratap_Singh_Delhi_4.00_yrs.docx
433 seo ShivaKumarG[4_11].docx
434 seo Avik_Bhattacharya_Bengaluru___Bangalore_2.07_yrs.pdf
435 seo SushantYM[2_7].docx
436 seo VikasRai[5_0].doc
437 seo PrincyJain[2_9].docx
438 seo Suvrangsu_Das_Kolkata_0.06_yrs.docx
439 seo KamalakarY[2_1].docx
440 seo TintuFrancis[7_0].doc
441 seo Tarun_Bhardwaj_Delhi_4.02_yrs.doc
442 seo Tanmoy_Basak_Kolkata_7.00_yrs.doc
443 seo Nikhil_Chandra_Reddy_Hyderabad___Secunderabad_3.00_yrs.doc
444 seo LakshmiVenugopal[4_0].doc
445 seo Balaji_D_Hyderabad___Secunderabad_8.02_yrs.docx
446 seo Nirup_S_Ran

In [6]:
len(label_nontech)

600

In [7]:

jdres_train_labels=[]
jdres_train_data=[]

for row in label_nontech:
    jdres_train_data.append(row[0])
    jdres_train_labels.append(row[1])
#print(jdres_train_data)
#print(len(jdres_train_data))

In [8]:
print(len(jdres_train_data))
print(len(jdres_train_labels))

600
600


In [12]:
df1 = pd.DataFrame(jdres_train_data,columns=['filecontent'])
df2 = pd.DataFrame(jdres_train_labels,columns=['label'])

In [13]:
#print (df2)

In [14]:
#print(jdres_train_data[42])

In [9]:
encoder = preprocessing.LabelEncoder()
encoder.fit(jdres_train_labels)
encoded_labels = encoder.transform(jdres_train_labels)
# convert integers to dummy variables (i.e. one hot encoded)
labels = to_categorical(encoded_labels,num_classes=4)
print(encoded_labels)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

In [10]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(jdres_train_data)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(jdres_train_data)
vocab_size

31679

In [11]:
# pad documents to a max length of 250 words
max_length = 1000
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[14590   166   121 ...,  1659   275   161]
 [ 5912   754  4115 ...,     0     0     0]
 [10355  2964 10355 ...,     0     0     0]
 ..., 
 [  285   349 31629 ...,     0     0     0]
 [ 2574 14586  2574 ...,     0     0     0]
 [ 5228  3864  3654 ...,     0     0     0]]


In [12]:
print('Shape of data tensor:', padded_docs.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (600, 1000)
Shape of label tensor: (600, 4)


In [13]:
indices = np.arange(padded_docs.shape[0])
np.random.shuffle(indices)
padded_docs = padded_docs[indices]
labels = labels[indices]

In [14]:
#loading glove

embeddings_index = dict()
f = open('/home/shabna/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [15]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [16]:
#create model

from keras.layers import Activation, Input, Dense, Flatten, Dropout, Embedding
from keras.layers import Conv1D, MaxPooling1D,GlobalMaxPooling1D
from keras.models import Model

In [17]:
#from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [18]:
def create_model():
    model = Sequential()
    e = Embedding(vocab_size,100, weights=[embedding_matrix], input_length=1000, trainable='true')
    model.add(e)
    model.add(Conv1D(128,5, activation='relu',name='l1'))
    model.add(MaxPooling1D(pool_size=5,name='l2'))
    model.add(Conv1D(128, 5, activation='relu',name='l3'))
    model.add(GlobalMaxPooling1D(name='l4'))
   
    model.add(Dense(12, activation='sigmoid',name='l5'))
    model.add(Dropout(0.2))
    #model.add(Flatten())
    model.add(Dense(4, activation='softmax',name='l6'))
    #adam = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return model

In [19]:
#model = Model(embedding_matrix, preds)
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 100)         3167900   
_________________________________________________________________
l1 (Conv1D)                  (None, 996, 128)          64128     
_________________________________________________________________
l2 (MaxPooling1D)            (None, 199, 128)          0         
_________________________________________________________________
l3 (Conv1D)                  (None, 195, 128)          82048     
_________________________________________________________________
l4 (GlobalMaxPooling1D)      (None, 128)               0         
_________________________________________________________________
l5 (Dense)                   (None, 12)                1548      
_________________________________________________________________
dropout_1 (Dropout)          (None, 12)                0         
__________

In [ ]:
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

param_grid = dict(learn_rate=learn_rate, momentum=momentum, batch_size=batch_size, epochs=epochs,optimizer=optimizer)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
    grid_result = grid.fit(X_train, y_train,validation_data=(X_val,y_val))
    
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [20]:
from keras.callbacks import ModelCheckpoint

In [21]:
from keras.callbacks import Callback

In [22]:
filepath="/home/shabna/Desktop/example_codes/weights_3_2.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only="True", mode="max")
callbacks_list = [checkpoint]

In [23]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

XX_train, X_test, yy_train, y_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=42)

i=1
for train_index, test_index in kf.split(XX_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    print(i,":num of fold")

    X_train, X_val = XX_train[train_index], XX_train[test_index]
    y_train, y_val = yy_train[train_index], yy_train[test_index]
    
    history = model.fit(X_train, y_train, validation_data=(X_val,y_val) ,epochs=50, batch_size=100, verbose=1, callbacks=callbacks_list)
   
    i=i+1
    

1 :num of fold
Train on 384 samples, validate on 96 samples
Epoch 1/50
384/384 [==============================] - 48s 124ms/step - loss: 1.4728 - acc: 0.2708 - val_loss: 1.4209 - val_acc: 0.2396
Epoch 2/50
384/384 [==============================] - 30s 77ms/step - loss: 1.3479 - acc: 0.3333 - val_loss: 1.3552 - val_acc: 0.3958
Epoch 3/50
384/384 [==============================] - 30s 79ms/step - loss: 1.3020 - acc: 0.3984 - val_loss: 1.3371 - val_acc: 0.3125
Epoch 4/50
384/384 [==============================] - 30s 79ms/step - loss: 1.2438 - acc: 0.6328 - val_loss: 1.2707 - val_acc: 0.6979
Epoch 5/50
384/384 [==============================] - 30s 77ms/step - loss: 1.1559 - acc: 0.7578 - val_loss: 1.2463 - val_acc: 0.4896
Epoch 6/50
384/384 [==============================] - 30s 77ms/step - loss: 1.0919 - acc: 0.7474 - val_loss: 1.1858 - val_acc: 0.8021
Epoch 7/50
384/384 [==============================] - 30s 78ms/step - loss: 1.0673 - acc: 0.7292 - val_loss: 1.1277 - val_acc: 0.7604
E

Epoch 12/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2245 - acc: 0.9870 - val_loss: 0.1689 - val_acc: 1.0000
Epoch 13/50
384/384 [==============================] - 30s 77ms/step - loss: 0.2167 - acc: 0.9896 - val_loss: 0.1661 - val_acc: 1.0000
Epoch 14/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2318 - acc: 0.9922 - val_loss: 0.1635 - val_acc: 1.0000
Epoch 15/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2080 - acc: 0.9948 - val_loss: 0.1607 - val_acc: 1.0000
Epoch 16/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2168 - acc: 0.9870 - val_loss: 0.1579 - val_acc: 1.0000
Epoch 17/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2037 - acc: 0.9922 - val_loss: 0.1554 - val_acc: 1.0000
Epoch 18/50
384/384 [==============================] - 30s 78ms/step - loss: 0.2423 - acc: 0.9844 - val_loss: 0.1528 - val_acc: 1.0000
Epoch 19/50
384/384 [==============================] - 

Epoch 45/50
384/384 [==============================] - 30s 79ms/step - loss: 0.1309 - acc: 1.0000 - val_loss: 0.0881 - val_acc: 1.0000
Epoch 46/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1237 - acc: 0.9948 - val_loss: 0.0868 - val_acc: 1.0000
Epoch 47/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1171 - acc: 0.9974 - val_loss: 0.0856 - val_acc: 1.0000
Epoch 48/50
384/384 [==============================] - 30s 77ms/step - loss: 0.1229 - acc: 0.9896 - val_loss: 0.0849 - val_acc: 1.0000
Epoch 49/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1266 - acc: 0.9974 - val_loss: 0.0850 - val_acc: 1.0000
Epoch 50/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1261 - acc: 1.0000 - val_loss: 0.0841 - val_acc: 1.0000
3 :num of fold
Train on 384 samples, validate on 96 samples
Epoch 1/50
384/384 [==============================] - 30s 78ms/step - loss: 0.1243 - acc: 1.0000 - val_loss: 0.0841 - val_acc: 1.0

384/384 [==============================] - 30s 78ms/step - loss: 0.0815 - acc: 1.0000 - val_loss: 0.0413 - val_acc: 1.0000
Epoch 10/50
384/384 [==============================] - 30s 78ms/step - loss: 0.0882 - acc: 0.9974 - val_loss: 0.0409 - val_acc: 1.0000
Epoch 11/50
384/384 [==============================] - 30s 78ms/step - loss: 0.0642 - acc: 1.0000 - val_loss: 0.0405 - val_acc: 1.0000
Epoch 12/50
384/384 [==============================] - 30s 78ms/step - loss: 0.0742 - acc: 0.9974 - val_loss: 0.0401 - val_acc: 1.0000
Epoch 13/50
384/384 [==============================] - 30s 78ms/step - loss: 0.0778 - acc: 1.0000 - val_loss: 0.0397 - val_acc: 1.0000
Epoch 14/50
384/384 [==============================] - 30s 78ms/step - loss: 0.0790 - acc: 0.9974 - val_loss: 0.0393 - val_acc: 1.0000
Epoch 15/50
384/384 [==============================] - 30s 78ms/step - loss: 0.0884 - acc: 0.9974 - val_loss: 0.0389 - val_acc: 1.0000
Epoch 16/50
384/384 [==============================] - 30s 77ms/ste

Epoch 42/50
384/384 [==============================] - 30s 77ms/step - loss: 0.0718 - acc: 0.9948 - val_loss: 0.0305 - val_acc: 1.0000
Epoch 43/50
384/384 [==============================] - 30s 78ms/step - loss: 0.0596 - acc: 0.9974 - val_loss: 0.0303 - val_acc: 1.0000
Epoch 44/50
384/384 [==============================] - 30s 78ms/step - loss: 0.0689 - acc: 0.9974 - val_loss: 0.0300 - val_acc: 1.0000
Epoch 45/50
384/384 [==============================] - 30s 78ms/step - loss: 0.0688 - acc: 0.9974 - val_loss: 0.0298 - val_acc: 1.0000
Epoch 46/50
384/384 [==============================] - 30s 78ms/step - loss: 0.0715 - acc: 0.9922 - val_loss: 0.0295 - val_acc: 1.0000
Epoch 47/50
384/384 [==============================] - 30s 78ms/step - loss: 0.0690 - acc: 0.9974 - val_loss: 0.0293 - val_acc: 1.0000
Epoch 48/50
384/384 [==============================] - 30s 78ms/step - loss: 0.0636 - acc: 1.0000 - val_loss: 0.0290 - val_acc: 1.0000
Epoch 49/50
384/384 [==============================] - 

In [4]:
model=load_model("weights_3_2.best.hdf5")


In [5]:
plot_model(model, to_file = 'models_3_2.png')

In [25]:
y_train

array([[ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.]])

In [26]:
from sklearn.metrics import classification_report, confusion_matrix

In [27]:
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred,axis=1)

In [28]:
y_pred = model.predict_classes(X_test)

120/120 [==============================] - 1s 12ms/step


In [29]:
p = model.predict_proba(X_test)

120/120 [==============================] - 1s 12ms/step


In [30]:
model.evaluate(X_test,y_test)

120/120 [==============================] - 2s 14ms/step


[0.35735614101092023, 0.95000000397364304]

In [31]:
model.evaluate(X_train,y_train)

384/384 [==============================] - 4s 12ms/step


[0.2635170804957549, 0.984375]

In [32]:
labels_index = {'hr':0 ,'marketing':1, 'other':2, 'seo':3}
target_name = [t for t in labels_index.keys()]

In [33]:
print(classification_report(np.argmax(y_test,axis=1),y_pred,target_names=target_name))

             precision    recall  f1-score   support

         hr       0.97      0.97      0.97        29
  marketing       1.00      0.86      0.93        37
      other       0.84      1.00      0.91        21
        seo       0.97      1.00      0.99        33

avg / total       0.96      0.95      0.95       120



In [34]:
print(confusion_matrix(np.argmax(y_test,axis=1),y_pred))

[[28  0  1  0]
 [ 1 32  3  1]
 [ 0  0 21  0]
 [ 0  0  0 33]]


In [35]:
from numpy import argmax
for i, item in enumerate (X_test):
    item= item.reshape(1, 1000)
    #print (item.shape)
    classes=model.predict(item)
    p=(argmax(classes),argmax(y_test[i]))
    #print(p)
    if (argmax(classes)!=argmax(y_test[i])):
        print(p)


(3, 1)
(2, 1)
(2, 0)
(2, 1)
(0, 1)
(2, 1)


In [282]:
X_test.shape


(120, 1000)

In [283]:
X_train.shape

(384, 1000)

In [284]:
print(X_test)

[[ 7166  6295 18068 ...,     0     0     0]
 [  139  5543   484 ...,     0     0     0]
 [   47    32 10290 ...,     0     0     0]
 ..., 
 [  139 19015 11794 ...,     0     0     0]
 [ 1392   121   139 ...,     0     0     0]
 [13212    29 26119 ...,     0     0     0]]


In [123]:
#predictions

parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level3/nontech/marketing/Imran_Sidi_Singapore_3.10_yrs.pdf")
parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level3/nontech/hr/ShrutiSharma[8_0].doc")

contents=[clean_doc(parsedPDF['content'])]
            
print(contents)

['personal information nationality indian email imransidihotmailcom mobile 65 8698 3868 languages english fluent hindi native french elementary summary global management student four years experience application technology management outsourcing imran’s skills technofunctional expert developed groomed moreover providing remote dedicated support services premier clients high pressure environments imran able hone excellent project management skills education 20152016 graduate april 2016 master business administration contemporary marketing management sp jain school global management singapore sydney dubai 2007–2011 bachelor technology electronics telecommunication mpstme – nmims university mumbai  india linkedin https wwwlinkedincominimransidi photo  delete box “paste over” please sidi imran professional experience accenture mumbai  melbourne july 2011 – may 2015 software engineering sr analyst  operation client – one largest mining company’s world ‘1 internet presence’ sharepoint 2010 b

In [124]:
def prpInput(pp):
    tx = Tokenizer()
    tx.fit_on_texts(pp)
    vocab_size = len(tx.word_index)+1
    encoded_doc = tx.texts_to_sequences(pp)
    max_length = 1000
    padded_docs = pad_sequences(encoded_doc, maxlen=max_length, padding='post')
    return padded_docs

In [125]:
mat=prpInput(contents)


In [126]:
prd = model.predict(mat)


In [127]:
#labels_index = {'marketing':0 ,'hr':1, 'seo':2, 'other':3}
rev_lable_index = {}
for key in labels_index:
    rev_lable_index[labels_index[key]] = key
print(rev_lable_index)
def result(prd,contents):
    y_classes = prd.argmax(axis=-1)
    print(len(y_classes))
    lx=[]
    for idx,lb in enumerate(y_classes):
        lx.append([contents[idx],rev_lable_index[lb]])
    return lx

{0: 'hr', 1: 'marketing', 2: 'other', 3: 'seo'}


In [128]:
result(prd,contents)

1


[['personal information nationality indian email imransidihotmailcom mobile 65 8698 3868 languages english fluent hindi native french elementary summary global management student four years experience application technology management outsourcing imran’s skills technofunctional expert developed groomed moreover providing remote dedicated support services premier clients high pressure environments imran able hone excellent project management skills education 20152016 graduate april 2016 master business administration contemporary marketing management sp jain school global management singapore sydney dubai 2007–2011 bachelor technology electronics telecommunication mpstme – nmims university mumbai  india linkedin https wwwlinkedincominimransidi photo  delete box “paste over” please sidi imran professional experience accenture mumbai  melbourne july 2011 – may 2015 software engineering sr analyst  operation client – one largest mining company’s world ‘1 internet presence’ sharepoint 2010 

In [129]:

pd.DataFrame(result(prd,contents),columns=['File Content','Label'])

1


,File Content,Label
0,personal information nationality indian email ...,hr


# Predictions


In [489]:
def test_load():
    i=0
    test=[]
    for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level3/testmix/"):
        print (i, file)
        parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level3/testmix/"+file)
        content = clean_doc(parsedPDF['content'])
        test.append(content)
        #mat=prpInput(contents)
        #prd = model.predict(mat)
        #print(result(prd,contents))
        #pd.DataFrame(result(prd,contents),columns=['File Content','Label'])
        i=i+1
        
    return test
testinput = test_load()

0 Ramya_nair_hr.docx
1 RajaVenkat[2_3].docx
2 ANAZ_A_N.pdf
3 Rasal_hr.pdf
4 Manjumol_Saleesh_Bengaluru_mrkt.docx
5 PRAKASHB_hr.doc
6 Pranav_hr.doc
7 Premkum_hr.doc
8 RajashriGHegde[1_10].doc
9 PremSing_hr.pdf
10 Laxmi_Kanth_Hyderabad__mrkt.docx


In [490]:
h=prpInput(testinput)

In [491]:
h

array([[1121, 1122, 1123, ...,    0,    0,    0],
       [ 762,  540, 1226, ...,    0,    0,    0],
       [1354,  567, 1355, ...,    0,    0,    0],
       ..., 
       [2089, 2090,  832, ...,    0,    0,    0],
       [   4,  274,  711, ...,   94,    3, 2373],
       [2374, 2375,  671, ...,    0,    0,    0]], dtype=int32)

In [492]:
prdi = model.predict(h)

In [493]:
rev_lable_index = {}
for key in labels_index:
    rev_lable_index[labels_index[key]] = key
print(rev_lable_index)
def result(prdi,testinput):
    y_classes = prdi.argmax(axis=-1)
    print("num of target items:",len(y_classes))
    lx=[]
    for idx,lb in enumerate(y_classes):
        lx.append([testinput[idx],rev_lable_index[lb]])
    return lx

{0: 'hr', 1: 'marketing', 2: 'other', 3: 'seo'}


In [494]:
result(prdi,testinput)

num of target items: 11


[['yashmin bhatta ramya nair krishna jyothi house near iravichira shiva temple iringole perumbavoor ernakulam contact 9446936288 email ramyavnair231gmailcom mba hr experience nearly 2 years hr operations recruitment  event management profile abridgement enhance knowledge tally 90 mis possessing knowledge interviewing assessing people expert recruiting people according company policy proficient giving valuable decision taking feedback employees core competencies endtoend recruitment  event management employee engagement freelance recruitments work lead team organisational experience 1 la associates perumbavoor la associates established 2013 provides total recruitment solution industrys man power requirements ranges client services include executive search  selection recruitment process outsourcing impeccable speed change technology keeping pace time indomitable challenge la associates helps making staffing requirements perceivable technical filed highly qualified experienced team consul

In [495]:
import pandas as pd
pd.DataFrame(result(prdi,testinput),columns=['File Content','Label'])

num of target items: 11


,File Content,Label
0,yashmin bhatta ramya nair krishna jyothi house...,marketing
1,raja v no11 saraswathy colony email rajavenkat...,marketing
2,indeed resume anaz n kollam kerala anaskrkngma...,hr
3,microsoft word rasal hameed cv newdoc page 1 ...,hr
4,manjumol 876 2nd floor 7th cross manorayanapla...,marketing
5,prakash b address sreenandanam nelliyode tc 64...,marketing
6,plot pranav kumar achar contact 91994900196491...,hr
7,academic qualifications profile enthusiastic r...,hr
8,ref srew0406 curriculum vitae rajashri g hegde...,marketing
9,resume prem singh email vickslivegmailcom cont...,marketing
